In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import lightkurve as lk
from modules import *
from astropy.table import Table

plt.style.use('seaborn-v0_8-darkgrid')

TIC = 'TIC 219107776'

In [ ]:
lcs = get_lightcurves(TIC, author = 'SPOC', cadence='short', use_till=2)


In [ ]:
new_lcs = []

for lc in lcs:
    # new_lc = straight_lines(lc, cadence_magnifier= 4)


    df = pd.DataFrame({'time': lc.time.jd, 'flux': np.array(lc.flux, dtype='f')})
    df.dropna(inplace=True)
    lg = lk.LightCurve(time= df['time'], flux= df['flux'])
    new_lc = lg


    new_lc = apply_savgol_filter(new_lc.time.jd, new_lc.flux, want = 'lc', displaygraphs= True, window_length_for_remove=1500)
    new_lcs.append(new_lc)
    new_lc.plot()